这段PostgreSQL语句是用来创建一个名为`kdigo_stages`的表，用于根据KDIGO标准检查患者是否患有急性肾损伤（AKI）。以下是各部分的字段含义：

- `subject_id`, `hadm_id`, `stay_id`：患者和住院相关的标识符。
- `charttime`：记录的时间点。
- `creatinine`相关字段：包含肌酐测量和历史肌酐水平的信息。
  - `creat_low_past_7day`：过去7天内最低的肌酐水平。
  - `creat_low_past_48hr`：过去48小时内最低的肌酐水平。
  - `creat`：当前的肌酐水平。
- `aki_stage_creat`：基于肌酐水平确定的AKI阶段。
- `uo`相关字段：包含尿量相关的信息和AKI阶段。
  - `uo_rt_6hr`, `uo_rt_12hr`, `uo_rt_24hr`：不同时间窗口（6小时、12小时、24小时）内的尿量率。
  - `aki_stage_uo`：基于尿量确定的AKI阶段。
- `aki_stage_crrt`：基于CRRT确定的AKI阶段。
- `aki_stage`：综合肌酐、尿量和CRRT确定的AKI阶段（选择三者中最严重的作为患者的当前AKI阶段）。
- `aki_stage_smoothed`：为了减少不连续性，平滑处理后的AKI阶段。
  - **为什么需要平滑处理：** 生理参数（如肌酐和尿量）可能会出现短期波动，这些波动可能导致AKI阶段的评估在短时间内不稳定或频繁变化。这种不连续性可能会对临床决策产生误导。
  - **平滑处理的方法：** 通过使用窗口函数MAX()和OVER()，考虑过去6小时内最高的AKI阶段，来平滑处理短期内的波动。这样做可以提供一个更稳定和连续的AKI阶段评估，减少误诊的可能性，并为医生提供更准确的患者状况。

这个SQL语句通过结合多个指标来筛选和分类急性肾损伤（AKI）患者，主要依据的是国际肾脏病学会（KDIGO）制定的AKI诊断和分级标准。KDIGO标准主要根据以下三个指标来诊断和分级AKI：

1. **肌酐水平（Creatinine Levels）**:
   - **3倍基线肌酐水平**：若当前肌酐水平是过去7天内最低肌酐的3倍或以上，判定为第3阶段AKI。
   - **肌酐水平≥4.0 mg/dL且有明显增加**：若肌酐水平达到或超过4.0 mg/dL，并且在过去48小时内增加了至少0.3 mg/dL或当前肌酐水平是过去7天内最低水平的1.5倍或以上，也判定为第3阶段AKI。
   - **2倍基线肌酐水平**：若当前肌酐水平是过去7天内最低肌酐的2倍或以上，判定为第2阶段AKI。
   - **肌酐水平轻微升高**：若当前肌酐水平比过去48小时内最低水平高出0.3 mg/dL或是过去7天内最低水平的1.5倍或以上，判定为第1阶段AKI。

2. **尿量（Urine Output）**:
   - **第3阶段**：24小时尿量小于0.3 ml/kg/h，或12小时内完全无尿。
   - **第2阶段**：12小时尿量小于0.5 ml/kg/h。
   - **第1阶段**：6至12小时尿量小于0.5 ml/kg/h。

3. **连续性肾脏替代治疗（CRRT）**:
   - 若患者开始进行CRRT治疗，通常表明其肾功能受到严重影响，因此在此标准下直接判定为第3阶段AKI。


In [1]:
import dask.dataframe as dd

aki_mimic = dd.read_csv('csv/aki_mimic.csv')

In [7]:
unique_count = aki_mimic['patientunitstayid'].shape[0].compute()
unique_aki_pat_ids = aki_mimic[aki_mimic['final_aki_status']!='No ICU Acquired AKI']['patientunitstayid'].nunique()
unique_aki_count = unique_aki_pat_ids.compute()
print(f"总共有 {unique_count} 名病人，其中有 {unique_aki_count} 患急性肾损伤（AKI）的病人")


总共有 104326 名病人，其中有 38131 患急性肾损伤（AKI）的病人
